In [1]:
import os
import numpy as np
import torch
import warnings

warnings.filterwarnings("ignore")
if os.getcwd()[-8:]=='examples':
    os.chdir('..')

os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Possible GPUS

In [2]:
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import importlib

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator

import src
import src.trainer as tr

### Settings

In [3]:
#### FOR DP
MAX_GRAD_NORM = 0.1
EPSILON = 2.0
DELTA = 1e-5

#### FOR TRAINING
LR = 2
EPOCHS = 40
BATCH_SIZE = 2048
MAX_PHYSICAL_BATCH_SIZE = 1024
RHO = 0.0

#### FOR SAVE
PATH = "./saved/"
NAME = "FMNIST_DPNAS_DPSGD_EPS2"
SAVE_PATH = PATH + NAME

### FOR MODELING
MODEL_NAME = "DPNASNet_FMNIST"
DATA = "FashionMNIST"
NORMALIZE = {'mean':[0.1307],
             'std':[0.3081]}
N_CLASSES = 10

### Data loader

In [4]:
data = src.Datasets(data_name=DATA, train_transform = transforms.ToTensor())
train_loader, test_loader = data.get_loader(batch_size=BATCH_SIZE, drop_last_train=False, num_workers=16)

Data Loaded!
Train Data Length : 60000
Test Data Length : 10000


### Model & Optimizer

In [5]:
#### Load model
model = src.utils.load_model(model_name=MODEL_NAME, n_classes=N_CLASSES).cuda() # Load model
model = ModuleValidator.fix(model)
ModuleValidator.validate(model, strict=False)

pytorch_total_params = sum(p.numel() for p in model.parameters())

print("model params: {:.4f}M".format(pytorch_total_params/1000000))

#### Define optimizer
optimizer = torch.optim.SGD(model.parameters(),lr=LR, momentum=0.9)

[(3, 0, 1), (0, 0, 2), (5, 1, 2), (4, 0, 3), (5, 1, 3), (3, 2, 3), (0, 0, 4), (0, 1, 4), (6, 2, 4), (3, 3, 4), (5, 0, 5), (6, 1, 5), (1, 2, 5), (4, 3, 5), (1, 4, 5)]
DPNASNet_FMNIST is loaded.
model params: 0.2146M


### Load PrivacyEngine from Opacus

In [6]:
privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

optimizer.target_epsilon = EPSILON
optimizer.target_delta = DELTA

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")
# 2.0664

rmodel = src.RobModel(model, n_classes=N_CLASSES, normalize=NORMALIZE).cuda()

Using sigma=2.63671875 and C=0.1


### Start Training

In [7]:
importlib.reload(tr)

trainer = tr.DpTrainer(NAME,rmodel)
trainer.max_physical_batch_size = MAX_PHYSICAL_BATCH_SIZE
trainer.record_rob(train_loader, test_loader)

In [8]:
trainer.fit(train_loader=train_loader, max_epoch=EPOCHS, start_epoch=0,
            optimizer=optimizer,
            scheduler=None, scheduler_type="Epoch",
            minimizer="DPSAT(rho={})".format(RHO),
            save_path=SAVE_PATH, save_best={"Clean(Val)":"HB"},
            save_type=None, save_overwrite=True, record_type="Epoch")

[FMNIST_DPNAS_DPSGD_EPS2]
Training Information.
-Epochs: 40
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 2
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
-Scheduler: None
-Save Path: ./saved/FMNIST_DPNAS_DPSGD_EPS2
-Save Type: None
-Record Type: Epoch
-Device: cuda:0
----------------------------------------------------------
Epoch   CALoss^p   Clean(Tr)   Clean(Val)   lr   Sec/it   
1       1.3652     70.7034     69.9000      2    0.1849   
----------------------------------------------------------
2       0.8021     76.8594     76.4000      2    0.3619   
----------------------------------------------------------
3       0.8676     79.6895     78.9300      2    0.3672   
----------------------------------------------------------
4       0.8226     81.6964     81.1000      2    0.3891   
----------------------------------------------------------
5       0.7728     82.9424     82.0500      2    0.4038   
------------------------------------------------------

### Evaluation

In [9]:
rmodel.load_dict(PATH+NAME+'/last.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 40), ('CALoss^p', 0.6156516009941697), ('Clean(Tr)', 88.22999399319228), ('Clean(Val)', 87.14), ('lr', 2)])


87.14

In [10]:
rmodel.load_dict(PATH+NAME+'/best.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 40), ('CALoss^p', 0.6156516009941697), ('Clean(Tr)', 88.22999399319228), ('Clean(Val)', 87.14), ('lr', 2)])


87.14